# ENGLISH TOP 20 QULAC

First start with english dataset just to set up the runs, do it on top 20 docs. For now we dont put any parameters, we just use the docs and the query and try to predict 1 claridying query.

In [ ]:
pip install datasets


In [6]:
def get_token_length(input_text):
    tokenized_input = tokenizer(input_text, padding=False, truncation=False)
    return len(tokenized_input["input_ids"])

# Calculate the average token length for all inputs
token_lengths = [get_token_length(entry["input"]) for entry in raw_data]

# Calculate the average and print it
average_token_length = sum(token_lengths) / len(token_lengths)
print(f"Average token length: {average_token_length}")

Average token length: 86448.01502504174


# SEQ2SEQ USING T5 TINY + top 5 docs

In [ ]:
pip install wandb

In [1]:
import wandb

# Paste your API key directly here
wandb.login(key="")


wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /users/Etu0/21402600/.netrc
wandb: Currently logged in as: hibaakbi (hibaakbi-sorbonne-universit-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
from transformers import AutoTokenizer
from datasets import Dataset
from sklearn.model_selection import train_test_split
import json

# Load your dataset
def tokenize_and_split_dataset(filename,modelname):
    with open(filename, "r", encoding="utf-8") as f:
        raw_data = json.load(f)
    
    # Split the raw data into train and validation sets
    train_data, eval_data = train_test_split(raw_data, test_size=0.1)
    
    # Convert the splits into Hugging Face Dataset objects
    train_dataset = Dataset.from_list(train_data)
    eval_dataset = Dataset.from_list(eval_data)
    
    # Choose your model
    model_name = modelname 
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # Tokenize the dataset
    def preprocess(batch):
        model_inputs = tokenizer(
            batch["input"],
            max_length=512,
            padding="max_length",
            truncation=True
        )
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(
                batch["output"],
                max_length=64,
                padding="max_length",
                truncation=True
            )
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs
    
    # Tokenize the dataset
    tokenized_train_dataset = train_dataset.map(preprocess, batched=True)
    tokenized_eval_dataset = eval_dataset.map(preprocess, batched=True)

    return tokenizer,tokenized_train_dataset,tokenized_eval_dataset




In [74]:
tokenizer,tokenized_train_dataset, tokenized_eval_dataset= tokenize_and_split_dataset("training_top5_qulac_PREPROCESSED_FOR_MODEL.json","google/t5-efficient-tiny")
print(tokenized_train_dataset[0])  # Optional: check one example

Map:   0%|          | 0/539 [00:00<?, ? examples/s]

/users/Etu0/21402600/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/60 [00:00<?, ? examples/s]

{'input': '[QUERY] I\'m looking for information on Rick Warren, the evangelical minister.\n[DOCUMENTS]\n[DOC 1] The XX Factor\\nBriefings\\nNews & Politics\\nArts\\nLife\\nBusiness & Tech\\nScience\\nPodcasts & Video\\nBlogs\\nThe XX Factor: Slate women blog about politics, etc...\\nHome\\nDialing Protective Services...\\nPosted\xa0Thursday, January 08, 2009 1:35 PM\\n| By Melinda Henneberger\\nEmily, I also agree with the Obama ban on strollers at the inauguration, but not because the crowd needs to be protected from babies (and their means of transportation). Here I am wondering whether it will be safe to take newly minted teenagers into the crowds that day\\u0014most parents I know are leaning against it\\u0014and toddlers would surely be at risk, no?\\nUpdate: Oops, now that I have read Hanna\'s post, I take it back: I\'m sure it will be fine!\\nFiled under: Obama, inauguration, strollers\\nOh, My Aching Back\\nPosted\xa0Thursday, January 08, 2009 1:18 PM\\n| By Hanna Rosin\\nEmily

In [ ]:
pip install evaluate

In [ ]:
pip install rouge_score

In [2]:
from huggingface_hub import login

login("")

In [3]:
from evaluate import load

bleu = load("bleu")
rouge = load("rouge")
meteor = load("meteor")

2025-05-12 23:11:33.397038: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-12 23:11:34.418103: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747084294.707124   30855 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747084294.782840   30855 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747084295.568091   30855 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [4]:
import numpy as np
import json
def compute_metrics(eval_pred):
    """
    This function calculates BLEU, ROUGE, and METEOR for the model predictions.
    eval_pred: tuple (predictions, references), where predictions are model outputs
    and references are the true target labels.
    """
    predictions, labels = eval_pred
    predictions= predictions[0]
    pred_ids = np.argmax(predictions, axis=-1) 
    # Decode the model's predicted tokens and true labels
    predicted_texts = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    true_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Calculate BLEU score
    bleu_score = bleu.compute(predictions=predicted_texts, references=true_texts)
    rouge_score = rouge.compute(predictions=predicted_texts, references=true_texts)
    meteor_score = meteor.compute(predictions=predicted_texts, references=true_texts)
   
    # Combine the metrics into a dictionary
    return {
        "bleu": bleu_score["bleu"],
        "rouge1": rouge_score["rouge1"],
        "rouge2": rouge_score["rouge2"],
        "rougeL": rouge_score["rougeL"],
        "meteor": meteor_score["meteor"]
    }




In [40]:

# import torch
device= torch.device("cpu")
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [42]:
from transformers import AutoModelForSeq2SeqLM, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import Dataset
import os
from tqdm import tqdm

def train(modelname, filename, tokenizer, tokenized_train_dataset, tokenized_eval_dataset, nb_epochs=30):
    model_name =  modelname
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name,from_tf=True)
    
    output_dir = "./results"
    logging_dir = os.path.join(output_dir, "logs")
    # Define training arguments
    training_args = TrainingArguments(
        output_dir="./results",  # Output directory for logs and checkpoints (optional)
        eval_strategy="epoch",  # Evaluate after every epoch
        save_strategy="epoch",
        learning_rate=5e-5,  # Learning rate
        per_device_train_batch_size=8,  # Batch size per device for training
        per_device_eval_batch_size=8,  # Batch size per device for evaluation
        num_train_epochs=nb_epochs,  # Number of training epochs
        logging_dir=logging_dir,
        use_cpu=True,
        logging_first_step=True,
        weight_decay=0.01,  # Weight decay for regularization
        logging_steps=50,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss", 
        greater_is_better=False,
        report_to="wandb",
        run_name=wandb.run.name
    )
    
    os.makedirs(logging_dir, exist_ok=True)
    
    # Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_eval_dataset,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,  # Pass the tokenizer for automatic tokenization during training
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
        
    )
    
    # Start the training (without saving)
    trainer.train()
    
    # You can evaluate the model here if you need
    # Example: Evaluate after training
    evaluation_results = trainer.evaluate()
    print(evaluation_results)
    
    
    predictions = trainer.predict(tokenized_eval_dataset)
    
    pred_ids = np.argmax(predictions.predictions[0], axis=-1)
    predicted_texts = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    true_texts = tokenizer.batch_decode(predictions.label_ids, skip_special_tokens=True)
    input_ids = tokenized_eval_dataset["input_ids"]
    input_texts = tokenizer.batch_decode(input_ids, skip_special_tokens=True)
    
    output_predictions_file = filename
    with open(output_predictions_file, "w", encoding="utf-8") as writer:
        for input_text, true_text, predicted_text in tqdm(zip(input_texts, true_texts, predicted_texts), total=len(true_texts)):
            output_dict = {
                "input": input_text,
                "true": true_text,
                "predicted": predicted_text
            }
            writer.write(json.dumps(output_dict) + "\n")
    
    print(f"Final predictions with inputs saved to {output_predictions_file}")

    wandb.finish()



In [77]:
wandb.init(
    project="T5-Clarifications",  
    name="T5 Tiny + TO5 DOCS",
    reinit=True                     
)
train("google/t5-efficient-tiny", "predicted_clarif_T5TINY_TOP5DOC.jsonl",tokenizer, tokenized_train_dataset, tokenized_eval_dataset,40)


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


/tmp/ipykernel_1123707/2720556636.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor
1,13.234300,1.517482,0.014952,0.211643,0.026537,0.205198,0.110794
2,4.799100,1.271452,0.000000,0.201595,0.017517,0.202409,0.115615
3,1.452900,1.084230,0.027275,0.260028,0.067534,0.259609,0.184927
4,1.326900,0.944666,0.053019,0.303056,0.096960,0.302544,0.248124
5,1.182600,0.820040,0.059990,0.323862,0.111305,0.322215,0.255717
6,1.011800,0.746734,0.061839,0.325240,0.105657,0.321485,0.263440
7,0.983700,0.705895,0.073363,0.342629,0.120120,0.337879,0.281048
8,0.891700,0.682155,0.084416,0.362100,0.130386,0.359398,0.303273
9,0.864700,0.668440,0.095278,0.377438,0.144101,0.371821,0.323610
10,0.837300,0.656724,0.101825,0.389156,0.154621,0.384049,0.325958


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


{'eval_loss': 0.5913133025169373, 'eval_bleu': 0.13806879921087598, 'eval_rouge1': 0.432904255543699, 'eval_rouge2': 0.19917965151874492, 'eval_rougeL': 0.4237685060228919, 'eval_meteor': 0.3912234163144579, 'eval_runtime': 0.9983, 'eval_samples_per_second': 60.105, 'eval_steps_per_second': 8.014, 'epoch': 40.0}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 67414.48it/s]

Final predictions with inputs saved to predicted_clarif_T5TINY_TOP5DOC.jsonl


eval/bleu,▂▁▂▄▄▄▅▅▆▆▆▆▇▇▆▇▆▇▇▇▇▇▇▇▇▇▇▇████████████
eval/loss,█▆▅▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/meteor,▁▁▃▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇██████████████████
eval/rouge1,▁▁▃▄▅▅▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇███████████████████
eval/rouge2,▁▁▃▄▅▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇████████████████
eval/rougeL,▁▁▃▄▅▅▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇███████████████████
eval/runtime,▂▂█▁▁▁▂▁▂▅▁▁▁▁▁▁▁▁▂▂▂▂▁▁▁▂▂▁▂▂▂▂▂▂▁▂▂▂▁▂
eval/samples_per_second,▇▇▁▇▇▇▇▇▇▃▇███████▆▇▇▇▇▇▇▇▇▇▇▇▇▆▇▇▇▇▇▇▇▇
eval/steps_per_second,▇▇▁▇▇▇▇▇▇▃▇███████▆▇▇▇▇▇▇▇▇▇▇▇▇▆▇▇▇▇▇▇▇▇
test/bleu,▁
test/loss,▁


# SEQ2SEQ T5 SMALL, + TOP5DOCS

In [104]:
tokenizer,tokenized_train_dataset, tokenized_eval_dataset= tokenize_and_split_dataset("training_top5_qulac_PREPROCESSED_FOR_MODEL.json","t5-small")
print(tokenized_train_dataset[0])  # Optional: check one example

Map:   0%|          | 0/539 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

{'input': '[QUERY] Find information about solar panels and companies that manufacture them.\n[DOCUMENTS]\n[DOC 1] Cooler Planet\\nCooler Planet\\nCooler Planet’s Blog\\nSkip to content\\n« Older posts\\nNow is a Great Time to Install Solar\xa0Power\\nJanuary 14, 2009 – 11:41 am\\nWhat, you say? The economy is in a shambles, two of my neighbors have been laid off and my heating bill is half again as much as it was last winter. How can this be a good time to install solar power?\\nLet\\u0019s take it step by step. The losses on Wall Street may impact the economy, but they are not the economy. Almost all the factories, farms, growers, and retailers who started 2008 are still around. More important, the cows, chickens, pigs, wheat and corn fields, cotton fields and forests \\u0013 the goods that make up the\\nreal\\neconomy \\u0013 are still growing.\\nYes, people are losing jobs in the banking and retail sectors, but these job losses are exacerbated by cutbacks in spending, leading to mor

In [105]:
wandb.init(
    project="T5-Clarifications",  
    name="T5 Small + TO5 DOCS"
)
train("t5-small", "predicted_clarif_T5SMALL_TOP5DOC.jsonl",tokenizer, tokenized_train_dataset, tokenized_eval_dataset,30)

(…)4339abf00c2d751f99a73c1c4b127be5d8a7c7d9:   0%|          | 0.00/242M [00:00<?, ?B/s]

W0000 00:00:1746409912.909384 1123707 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
All TF 2.0 model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_1123707/4023069370.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor
1,7.069100,0.771933,0.149806,0.404954,0.176795,0.393308,0.334293
2,0.969000,0.549294,0.174324,0.437280,0.207905,0.424256,0.386653
3,0.571200,0.470458,0.208231,0.497650,0.239241,0.489789,0.447375
4,0.530600,0.454921,0.191625,0.493873,0.227596,0.483214,0.435498
5,0.509200,0.447265,0.196918,0.506433,0.239022,0.494439,0.455201
6,0.486100,0.439352,0.208233,0.503904,0.245829,0.491247,0.453277
7,0.448400,0.434164,0.223288,0.517980,0.259163,0.509867,0.471452
8,0.451100,0.433264,0.223910,0.519740,0.259430,0.509617,0.472943
9,0.432900,0.432279,0.219700,0.513982,0.251678,0.503930,0.462324
10,0.432700,0.428119,0.221895,0.515922,0.256638,0.505348,0.461391


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


{'eval_loss': 0.4268746078014374, 'eval_bleu': 0.23130231703063925, 'eval_rouge1': 0.5250809284537804, 'eval_rouge2': 0.26727968353866854, 'eval_rougeL': 0.5172638348759238, 'eval_meteor': 0.4754573042921671, 'eval_runtime': 2.2709, 'eval_samples_per_second': 26.421, 'eval_steps_per_second': 3.523, 'epoch': 14.0}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 61380.06it/s]

Final predictions with inputs saved to predicted_clarif_T5SMALL_TOP5DOC.jsonl


eval/bleu,▁▃▆▅▅▆▇▇▇▇█████
eval/loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁
eval/meteor,▁▄▇▆▇▇██▇▇███▇█
eval/rouge1,▁▃▆▆▇▇██▇▇██▇▇█
eval/rouge2,▁▃▆▅▆▆▇▇▇▇██▇██
eval/rougeL,▁▃▆▆▇▇██▇▇██▇▇█
eval/runtime,██████▇▁▅▅▄▄▄▅▅
eval/samples_per_second,▁▁▁▁▁▁▂█▄▄▅▅▄▄▄
eval/steps_per_second,▁▁▁▁▁▁▂█▄▄▅▅▄▄▄
test/bleu,▁
test/loss,▁


## same run doen earlier before wandb -IGNORE
do it for 30 epoch

In [41]:
# from transformers import AutoModelForSeq2SeqLM, Trainer, TrainingArguments,EarlyStoppingCallback
# from datasets import Dataset
# import os


# model_name = "t5-small"
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# output_dir = "./results"
# logging_dir = os.path.join(output_dir, "logs")
# # Define training arguments
# training_args = TrainingArguments(
#     output_dir="./results",  # Output directory for logs and checkpoints (optional)
#     eval_strategy="epoch",  # Evaluate after every epoch
#     save_strategy="epoch",
#     learning_rate=5e-5,  # Learning rate
#     per_device_train_batch_size=16,  # Batch size per device for training
#     per_device_eval_batch_size=16,  # Batch size per device for evaluation
#     num_train_epochs=30,  # Number of training epochs
#     logging_dir=logging_dir,
#     use_cpu=True,
#     logging_first_step=True,
#     weight_decay=0.01,  # Weight decay for regularization
#     logging_steps=50,
#     load_best_model_at_end=True,
#     metric_for_best_model="eval_loss", 
#     greater_is_better=False
# )

# os.makedirs(logging_dir, exist_ok=True)

# # Initialize the Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_train_dataset,
#     eval_dataset=tokenized_eval_dataset,
#     compute_metrics=compute_metrics,
#     tokenizer=tokenizer,  # Pass the tokenizer for automatic tokenization during training
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
# )

# # Start the training (without saving)
# trainer.train()

# # You can evaluate the model here if you need
# # Example: Evaluate after training
# evaluation_results = trainer.evaluate()
# print(evaluation_results)



/tmp/ipykernel_674230/1610004705.py:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor
1,21.891900,0.822087,0.124927,0.377650,0.152483,0.372101,0.286300
2,6.389900,0.734556,0.141975,0.400126,0.167344,0.385960,0.343383
3,0.887300,0.604645,0.167292,0.434119,0.207284,0.424159,0.387383
4,0.887300,0.501504,0.186885,0.457414,0.221770,0.449804,0.413395
5,0.656200,0.471018,0.194042,0.497377,0.225994,0.490047,0.437352
6,0.541100,0.461405,0.199731,0.495848,0.231933,0.489790,0.445434
7,0.541100,0.455307,0.197389,0.493090,0.232453,0.489384,0.441052
8,0.493800,0.449578,0.193624,0.504263,0.233002,0.494275,0.448787
9,0.483700,0.446134,0.198189,0.506828,0.245310,0.500292,0.455046
10,0.483700,0.442523,0.208601,0.510335,0.255664,0.504310,0.463587


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


{'eval_loss': 0.43086084723472595, 'eval_bleu': 0.22540464040675906, 'eval_rouge1': 0.5190223773296081, 'eval_rouge2': 0.26161976710516216, 'eval_rougeL': 0.5105622466603756, 'eval_meteor': 0.4665770243147761, 'eval_runtime': 2.1916, 'eval_samples_per_second': 27.377, 'eval_steps_per_second': 1.825, 'epoch': 25.0}


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (2, 60) + inhomogeneous part.

In [44]:
# predictions = trainer.predict(tokenized_eval_dataset)

# pred_ids = np.argmax(predictions.predictions[0], axis=-1)
# predicted_texts = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
# true_texts = tokenizer.batch_decode(predictions.label_ids, skip_special_tokens=True)

# output_predictions_file = "predicted_clarif_TINYT5_30EPOCH.jsonl"
# with open(output_predictions_file, "w", encoding="utf-8") as writer:
#     for true_text, predicted_text in zip(true_texts, predicted_texts):
#         output_dict = {"true": true_text, "predicted": predicted_text}
#         writer.write(json.dumps(output_dict) + "\n")

# print(f"Final predictions saved to {output_predictions_file}")

Final predictions saved to predicted_clarif_TINYT5_30EPOCH.jsonl


# SE2SEQ T5 SMALL + TO5 DOCS RUN ON WANDB - just ignore set up wasnt finished

In [67]:
#WANDB
train("t5-small", "predicted_clarif_T5SMALL_TOP5DOC_WNDB.jsonl","T5-SMALL TOP5DOCS",tokenizer, tokenized_train_dataset, tokenized_eval_datset,30)

/tmp/ipykernel_1123707/1094267103.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor
1,6.781100,0.812190,0.117157,0.391130,0.181558,0.377999,0.326605
2,0.946100,0.601260,0.154446,0.424232,0.224240,0.417415,0.387250
3,0.596800,0.516339,0.169055,0.465668,0.233193,0.457669,0.422066
4,0.527900,0.501994,0.157821,0.473764,0.230911,0.465590,0.434874
5,0.496200,0.494720,0.160844,0.466397,0.226154,0.458146,0.430444
6,0.460600,0.490341,0.165680,0.480229,0.240972,0.471348,0.439784
7,0.466800,0.483008,0.164266,0.481503,0.240175,0.474668,0.445833
8,0.432900,0.483343,0.166211,0.481178,0.234927,0.473599,0.443244
9,0.433700,0.480551,0.165734,0.478390,0.231603,0.470114,0.439596
10,0.414700,0.480789,0.171579,0.496970,0.243226,0.486260,0.455332


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


{'eval_loss': 0.4787222743034363, 'eval_bleu': 0.17243909861428214, 'eval_rouge1': 0.49321804168360456, 'eval_rouge2': 0.24059370319927537, 'eval_rougeL': 0.4801302646389882, 'eval_meteor': 0.44887556862862926, 'eval_runtime': 2.2903, 'eval_samples_per_second': 26.197, 'eval_steps_per_second': 3.493, 'epoch': 14.0}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 73930.15it/s]

Final predictions with inputs saved to predicted_clarif_T5SMALL_TOP5DOC_WNDB.jsonl


# SE2SEQ T5 SMALL + QUERY ONLY 

In [106]:
tokenizer,tokenized_train_dataset, tokenized_eval_dataset= tokenize_and_split_dataset("training_queryonly_qulac_PREPROCESSED_FOR_MODEL.json","t5-small")
print(tokenized_train_dataset[0])  # Optional: check one example

Map:   0%|          | 0/539 [00:00<?, ? examples/s]

/users/Etu0/21402600/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/60 [00:00<?, ? examples/s]

{'input': "I'm looking for information on Rick Warren, the evangelical minister.", 'output': "I'm looking for background and biographical information on Rick Warren.", 'input_ids': [27, 31, 51, 479, 21, 251, 30, 11066, 16700, 6, 8, 30682, 6323, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [107]:
#WANDB
wandb.init(
    project="T5-Clarifications",  
    name="T5 Small + Query ONLY"
)
train("t5-small", "predicted_clarif_T5SMALL_queryonly_WNDB.jsonl",tokenizer, tokenized_train_dataset, tokenized_eval_dataset,30)

All TF 2.0 model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_1123707/4023069370.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor
1,4.926500,0.909643,0.110387,0.357508,0.179116,0.348935,0.296328
2,1.184500,0.728811,0.120721,0.389918,0.193348,0.380868,0.332936
3,0.716800,0.575246,0.151035,0.434860,0.223656,0.426650,0.393620
4,0.608900,0.536134,0.153890,0.447716,0.224096,0.435802,0.406838
5,0.567000,0.521996,0.156953,0.453647,0.224420,0.442497,0.411973
6,0.520900,0.513575,0.161386,0.462814,0.233679,0.452933,0.419951
7,0.519400,0.507583,0.164982,0.459551,0.236061,0.454089,0.420570
8,0.483600,0.504109,0.158770,0.457573,0.223498,0.449212,0.415757
9,0.485600,0.500099,0.160056,0.467163,0.233950,0.459600,0.427607
10,0.462600,0.498216,0.159606,0.466377,0.235319,0.455765,0.424606


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


{'eval_loss': 0.48929399251937866, 'eval_bleu': 0.17286262087859694, 'eval_rouge1': 0.48174823070272255, 'eval_rouge2': 0.24570588499476026, 'eval_rougeL': 0.4669216196461488, 'eval_meteor': 0.4379267959319926, 'eval_runtime': 1.9192, 'eval_samples_per_second': 31.263, 'eval_steps_per_second': 4.168, 'epoch': 23.0}


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 163096.72it/s]

Final predictions with inputs saved to predicted_clarif_T5SMALL_queryonly_WNDB.jsonl


eval/bleu,▁▂▅▆▆▇▇▆▆▆▆▆▇███████████
eval/loss,█▅▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/meteor,▁▃▆▆▇▇▇▇▇▇▇▇▇█▇▇█▇██████
eval/rouge1,▁▃▅▆▆▇▇▆▇▇▇▇▇▇▇▇████████
eval/rouge2,▁▂▅▅▆▆▇▅▆▇▆▆▇█▇▇█▇▇█▇███
eval/rougeL,▁▃▆▆▆▇▇▇▇▇▇▇▇██▇████████
eval/runtime,██████▂▁▁▁▂▁▁▁▂▁▂▂▁▁▁▃▁▂
eval/samples_per_second,▁▁▁▁▁▁▇▇▇█▇██▇▇█▇▇▇█▇▆▇▇
eval/steps_per_second,▁▁▁▁▁▁▇▇▇█▇███▇█▇▇▇█▇▆▇▇
test/bleu,▁
test/loss,▁


# SEQ2SEQ T5 SMALL + TOP 5 SUMMARIZED DOCS

In [109]:
tokenizer,tokenized_train_dataset, tokenized_eval_dataset= tokenize_and_split_dataset("training_top5_qulac_SUMMARIZED_FOR_MODEL.json","t5-small")
print(tokenized_train_dataset[0])  # Optional: check one example

Map:   0%|          | 0/539 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

{'input': "[QUERY] Find information about solar panels and companies that manufacture them.\n[DOCUMENTS]\n[DOC 1] Incentives are incentives for people who want to make money from incentives . The incentives for incentives are still relevant in the United States .\n[DOC 2] Online Solar, Inc. & Mr.Solar.com offer solar panels . We know solar panels!\n[DOC 3] A criminal defense attorney, a criminal defense lawyer and a home loaner, has been featured on this blog . The site is designed by Bugdugle Blog Design .\n[DOC 4] Xantrex XW Solar Charge Controller MPPT 865-1030Xantrex Cm/R-50 Digitalmeter-C12/35/40/60-50'30' MC1 Connector #10 AWG Male/Female .\n[DOC 5] Solar Technology Company (IST) has developed a concentrator system that incorporates the reflective surface as part of the collector structure . This results in a very lightweight, low-cost concentrator module that is also very strong .", 'output': 'Find information about nano-technological solar power.', 'input_ids': [784, 15367, 118

In [110]:
wandb.init(
    project="T5-Clarifications",  
    name="T5 SMALL + TO5 DOCS SUMMARIZED"
)
train("t5-small", "predicted_clarif_T5SMALL_TOP5DOC_SUMMARIZED.jsonl",tokenizer, tokenized_train_dataset, tokenized_eval_dataset,30)

All TF 2.0 model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_1123707/4023069370.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor
1,5.934200,0.742373,0.124662,0.386160,0.183804,0.374544,0.311506
2,0.977200,0.523263,0.181451,0.448648,0.241939,0.443039,0.399988
3,0.551300,0.465223,0.189024,0.471199,0.237287,0.468122,0.441595
4,0.523500,0.450799,0.199623,0.482653,0.245469,0.477889,0.452465
5,0.506200,0.444649,0.197712,0.483458,0.248663,0.480008,0.452078
6,0.482700,0.435452,0.205548,0.496380,0.261015,0.491287,0.461446
7,0.438000,0.432831,0.219565,0.510013,0.267423,0.504820,0.475660
8,0.446300,0.431656,0.226739,0.513442,0.271191,0.508927,0.472623
9,0.431700,0.428373,0.233349,0.517839,0.272771,0.512893,0.478949
10,0.428700,0.426955,0.236857,0.522491,0.278750,0.516506,0.480049


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


{'eval_loss': 0.42411214113235474, 'eval_bleu': 0.22573487255751198, 'eval_rouge1': 0.5185140104023928, 'eval_rouge2': 0.2665282907380432, 'eval_rougeL': 0.5102561977502414, 'eval_meteor': 0.47725188098889065, 'eval_runtime': 1.9468, 'eval_samples_per_second': 30.819, 'eval_steps_per_second': 4.109, 'epoch': 16.0}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 74411.07it/s]

Final predictions with inputs saved to predicted_clarif_T5SMALL_TOP5DOC_SUMMARIZED.jsonl


eval/bleu,▁▄▅▆▅▆▇▇████▇███▇
eval/loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/meteor,▁▄▆▆▆▇▇▇██▇█▇███▇
eval/rouge1,▁▄▅▆▆▇▇▇█████████
eval/rouge2,▁▅▅▅▆▇▇▇▇█▇█▇█▇█▇
eval/rougeL,▁▄▆▆▆▇▇▇█████████
eval/runtime,▅▅▅▅▆▅▅▅▆▅▄▂▂█▁▂▅
eval/samples_per_second,▄▄▄▄▃▄▄▄▃▄▅▇▇▁█▇▄
eval/steps_per_second,▃▄▄▄▃▄▄▄▃▄▅▇▇▁█▇▄
test/bleu,▁
test/loss,▁


# SEQ2SEQ T5 BASE + TOP 5 SUMMARIZED DOCS

In [112]:
tokenizer,tokenized_train_dataset, tokenized_eval_dataset= tokenize_and_split_dataset("training_top5_qulac_SUMMARIZED_FOR_MODEL.json","t5-base")
print(tokenized_train_dataset[0])  # Optional: check one example

Map:   0%|          | 0/539 [00:00<?, ? examples/s]

/users/Etu0/21402600/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/60 [00:00<?, ? examples/s]

{'input': "[QUERY] I'm looking for information on Rick Warren, the evangelical minister.\n[DOCUMENTS]\n[DOC 1] Elizabeth Alexander is an African-American, born in Harlem in 1962, who has published four books . Her work speaks about black experience, but her poems work more at being complex than didactic . Her best poems are imaginatively expansive as well as philosophical .\n[DOC 2] Presenting Warren front and centre as part of the first official act of the Obama Administration is not just wrong, it is also stupid . For a world longing for the end of extra legal policies that Bush pursued .\n[DOC 3] The Rev. Moon is invited to introduce Barack Obama to god on January 20 . Atheists say Obama is inviting Warren not because he supports his kind of theocracy, but because he doesn't .\n[DOC 4] Blogs include: A Guy in the Pew, A Lie A Day, A Religious Liberal Blog, A Blog of the Greateful Bear, The Rev. Rev. Susan Brooks Thistlethwaite, The Revealer, Theolog: The Blog of The Christian Centur

In [115]:
wandb.init(
    project="T5-Clarifications",  
    name="T5 BASE + TO5 DOCS SUMMARIZED"
)
train("t5-base", "predicted_clarif_T5BASE_TOP5DOC_SUMMARIZED.jsonl",tokenizer, tokenized_train_dataset, tokenized_eval_dataset,30)

tf_model.h5:   0%|          | 0.00/892M [00:00<?, ?B/s]

All TF 2.0 model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/tmp/ipykernel_1123707/4023069370.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor
1,2.482500,0.445160,0.201440,0.492116,0.263184,0.485924,0.454750
2,0.490100,0.386635,0.220864,0.535610,0.299028,0.523957,0.499894
3,0.381200,0.380820,0.225631,0.534435,0.300286,0.524182,0.501660
4,0.341100,0.372675,0.223258,0.541405,0.290740,0.528428,0.505760
5,0.321100,0.376765,0.235075,0.537244,0.293451,0.522374,0.508591
6,0.279500,0.383681,0.235180,0.529471,0.287502,0.515008,0.503549
7,0.271100,0.392567,0.226530,0.517833,0.275241,0.504018,0.490656


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


{'eval_loss': 0.3726748824119568, 'eval_bleu': 0.22325809117239986, 'eval_rouge1': 0.5414049646830954, 'eval_rouge2': 0.2907402855429171, 'eval_rougeL': 0.5284277816775549, 'eval_meteor': 0.5057596460391415, 'eval_runtime': 5.6346, 'eval_samples_per_second': 10.648, 'eval_steps_per_second': 1.42, 'epoch': 7.0}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 69060.99it/s]

Final predictions with inputs saved to predicted_clarif_T5BASE_TOP5DOC_SUMMARIZED.jsonl


eval/bleu,▁▅▆▆██▆▆
eval/loss,█▂▂▁▁▂▃▁
eval/meteor,▁▇▇██▇▆█
eval/rouge1,▁▇▇█▇▆▅█
eval/rouge2,▁██▆▇▆▃▆
eval/rougeL,▁▇▇█▇▆▄█
eval/runtime,▂▅▃▃▁▇▂█
eval/samples_per_second,▇▄▆▆█▂▇▁
eval/steps_per_second,▇▃▆▆█▂▇▁
test/bleu,▁
test/loss,▁


# TFIDF DOC REP + T5 SMALL

In [46]:
tokenizer,tokenized_train_dataset, tokenized_eval_dataset= tokenize_and_split_dataset("training_top5_qulac_PREPROCESSED_FOR_MODEL_TFIDF.json","t5-small")
print(tokenized_train_dataset[0])  # Optional: check one example

Map:   0%|          | 0/539 [00:00<?, ? examples/s]

/users/Etu0/21402600/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/60 [00:00<?, ? examples/s]

{'input': "[QUERY] I'm looking for information on Rick Warren, the evangelical minister.\n[DOCUMENTS]\n[DOC 1] obama warren inauguration rick nfiled barack prayer wright nposted jeremiah u0014and u0019s alexander dahlia u0019t gay church inaugural hitchens hanna\n[DOC 2] obama warren pmreply lgbt bilerico rick comment nbil december ni community nwe ve hate browning bil inauguration 2008 say prayer\n[DOC 3] obama u0014 nwed nlogin reply warren register rick 12 17 2008 barack hearts nre nthu 14 progressives progressive angelfoodmcspade invocation\n[DOC 4] warren rick inauguration obama rev currie nucc church wrong voice christ progressive blog religious ucc chuck portland nfaith parkrose oregon\n[DOC 5] 17th et pm obama 2008 warren gay invocation rick evangelical people ni nobama god choice president nrick bible barack evangelicals", 'output': "I'm looking for background and biographical information on Rick Warren.", 'input_ids': [784, 15367, 11824, 908, 27, 31, 51, 479, 21, 251, 30, 110

In [47]:
wandb.init(
    project="T5-Clarifications",  
    name="T5 SMALL + TO5 DOCS TFIDF v2. "                   
)
train("t5-small", "predicted_clarif_T5SMALL_TOP5DOC_TFIDF_v2.jsonl",tokenizer, tokenized_train_dataset, tokenized_eval_dataset,40)


All TF 2.0 model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_1389011/287362180.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor
1,5.430100,0.748300,0.126248,0.397713,0.194673,0.388640,0.340054
2,0.908400,0.568506,0.146624,0.430685,0.214592,0.423728,0.378188
3,0.598700,0.517328,0.163846,0.460022,0.229027,0.450214,0.416359
4,0.532000,0.501637,0.163275,0.463841,0.227477,0.453015,0.416833
5,0.508400,0.493275,0.169298,0.462386,0.233447,0.455440,0.417341
6,0.466000,0.486814,0.178139,0.473974,0.240766,0.466193,0.433736
7,0.469000,0.481031,0.180324,0.480001,0.259340,0.474954,0.443831
8,0.436300,0.480974,0.190290,0.487991,0.264242,0.481337,0.448620
9,0.436600,0.478462,0.187314,0.490617,0.263446,0.482397,0.452956
10,0.416800,0.476689,0.180338,0.483891,0.257500,0.476725,0.449168


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


{'eval_loss': 0.47529616951942444, 'eval_bleu': 0.18737064277859514, 'eval_rouge1': 0.49481248246904985, 'eval_rouge2': 0.26392867554129157, 'eval_rougeL': 0.4843460519697246, 'eval_meteor': 0.459091339323111, 'eval_runtime': 2.0702, 'eval_samples_per_second': 28.982, 'eval_steps_per_second': 3.864, 'epoch': 14.0}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 98419.34it/s]

Final predictions with inputs saved to predicted_clarif_T5SMALL_TOP5DOC_TFIDF_v2.jsonl


eval/bleu,▁▃▅▅▅▆▇█▇▇▇█▇█▇
eval/loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁
eval/meteor,▁▃▅▅▅▆▇▇█▇█████
eval/rouge1,▁▃▅▆▆▆▇▇▇▇█████
eval/rouge2,▁▃▄▄▅▅▇██▇██▇██
eval/rougeL,▁▄▅▆▆▇▇██▇█████
eval/runtime,▁▂▂▃▅▆▆▅▅█▄▅▇▄▄
eval/samples_per_second,█▇▇▆▄▃▃▄▄▁▅▄▂▅▅
eval/steps_per_second,█▇▇▆▄▃▃▄▄▁▅▄▂▅▅
test/bleu,▁
test/loss,▁


# TFIDF AVEC 20 DOCS

In [14]:
tokenizer,tokenized_train_dataset, tokenized_eval_dataset= tokenize_and_split_dataset("training_top20_qulac_PREPROCESSED_FOR_MODEL_TFIDF.json","t5-small")
print(tokenized_train_dataset[0])  # Optional: check one example

Map:   0%|          | 0/539 [00:00<?, ? examples/s]

/users/Etu0/21402600/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/60 [00:00<?, ? examples/s]

{'input': "[QUERY] I'm looking for information on Rick Warren, the evangelical minister.\n[DOCUMENTS]\n[DOC 1] obama warren inauguration rick nfiled barack prayer wright nposted jeremiah u0014and u0019s alexander dahlia u0019t gay church inaugural hitchens hanna\n[DOC 2] obama warren pmreply lgbt bilerico rick comment nbil december ni community nwe ve hate browning bil inauguration 2008 say prayer\n[DOC 3] obama u0014 nwed nlogin reply warren register rick 12 17 2008 barack hearts nre nthu 14 progressives progressive angelfoodmcspade invocation\n[DOC 4] warren rick inauguration obama rev currie nucc church wrong voice christ progressive blog religious ucc chuck portland nfaith parkrose oregon\n[DOC 5] 17th et pm obama 2008 warren gay invocation rick evangelical people ni nobama god choice president nrick bible barack evangelicals\n[DOC 6] obama flag warren dec reply nposted gay marriage civil nno roosevelt 23 gays rights washington 2008 japhy pm unions lgbt\n[DOC 7] warren church cri r

In [16]:
wandb.init(
    project="T5-Clarifications",  
    name="T5 SMALL + TOP 20 DOCS TFIDF "                   
)
train("t5-small", "predicted_clarif_T5SMALL_TOP20DOC_TFIDF.jsonl",tokenizer, tokenized_train_dataset, tokenized_eval_dataset,40)


All TF 2.0 model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_1389011/1914975726.py:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor
1,20.627000,3.924397,0.025245,0.355990,0.172432,0.347286,0.229766
2,20.627000,0.820348,0.104493,0.363480,0.161490,0.354000,0.291797
3,4.525100,0.759679,0.124591,0.390347,0.189394,0.379454,0.332368
4,4.525100,0.677909,0.151313,0.414139,0.206800,0.402357,0.359550
5,4.525100,0.624031,0.137112,0.408605,0.197611,0.401066,0.358688
6,0.791100,0.578247,0.143713,0.414690,0.204435,0.406755,0.366876
7,0.791100,0.535245,0.154061,0.436243,0.216300,0.427037,0.394729
8,0.791100,0.520411,0.151011,0.453964,0.221205,0.445504,0.408364
9,0.584800,0.511297,0.158875,0.464410,0.227780,0.452663,0.417638
10,0.584800,0.504855,0.159205,0.467033,0.227396,0.457840,0.422107


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

{'eval_loss': 0.47644302248954773, 'eval_bleu': 0.19160596633706883, 'eval_rouge1': 0.4934219280997374, 'eval_rouge2': 0.2688020232562909, 'eval_rougeL': 0.48405262705132923, 'eval_meteor': 0.45834750682011727, 'eval_runtime': 0.6782, 'eval_samples_per_second': 88.475, 'eval_steps_per_second': 2.949, 'epoch': 35.0}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 75709.46it/s]

Final predictions with inputs saved to predicted_clarif_T5SMALL_TOP20DOC_TFIDF.jsonl


eval/bleu,▁▄▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇███████████████
eval/loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/meteor,▁▃▄▅▅▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇████████████████
eval/rouge1,▁▁▃▄▄▄▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇███████████████
eval/rouge2,▂▁▃▄▃▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇███████████████
eval/rougeL,▁▁▃▄▄▄▅▆▆▇▇▇▇▇▇▇▇█▇▇████████████████
eval/runtime,▂▂▁▃▂▁█▂▂▂▂▂▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁
eval/samples_per_second,▆▇█▅▆█▁▆▇▇▇▆██▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▇▇█
eval/steps_per_second,▆▇█▅▆█▁▆▇▇▇▆██▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▇▇█
test/bleu,▁
test/loss,▁


# TEST AVEC TOP10 - TFIDF-

In [27]:
tokenizer,tokenized_train_dataset, tokenized_eval_dataset= tokenize_and_split_dataset("training_top10_qulac_PREPROCESSED_FOR_MODEL_TFIDF.json","t5-small")
print(tokenized_train_dataset[0])  # Optional: check one example

Map:   0%|          | 0/539 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

{'input': "[QUERY] I'm looking for information on Rick Warren, the evangelical minister.\n[DOCUMENTS]\n[DOC 1] obama warren inauguration rick nfiled barack prayer wright nposted jeremiah u0014and u0019s alexander dahlia u0019t gay church inaugural hitchens hanna\n[DOC 2] obama warren pmreply lgbt bilerico rick comment nbil december ni community nwe ve hate browning bil inauguration 2008 say prayer\n[DOC 3] obama u0014 nwed nlogin reply warren register rick 12 17 2008 barack hearts nre nthu 14 progressives progressive angelfoodmcspade invocation\n[DOC 4] warren rick inauguration obama rev currie nucc church wrong voice christ progressive blog religious ucc chuck portland nfaith parkrose oregon\n[DOC 5] 17th et pm obama 2008 warren gay invocation rick evangelical people ni nobama god choice president nrick bible barack evangelicals\n[DOC 6] obama flag warren dec reply nposted gay marriage civil nno roosevelt 23 gays rights washington 2008 japhy pm unions lgbt\n[DOC 7] warren church cri r

In [28]:
wandb.init(
    project="T5-Clarifications",  
    name="T5 SMALL + TOP 10 DOCS TFIDF "                   
)
train("t5-small", "predicted_clarif_T5SMALL_TOP10DOC_TFIDF.jsonl",tokenizer, tokenized_train_dataset, tokenized_eval_dataset,40)


All TF 2.0 model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_1389011/287362180.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor
1,5.056500,0.714510,0.130299,0.393877,0.195397,0.382513,0.342864
2,0.879100,0.564642,0.146328,0.435212,0.206842,0.424295,0.386815
3,0.599400,0.513678,0.153190,0.446186,0.211427,0.437752,0.398847
4,0.533400,0.496594,0.158607,0.464045,0.223636,0.454576,0.418017
5,0.507500,0.488396,0.164677,0.469698,0.229723,0.460327,0.424169
6,0.465700,0.482871,0.163358,0.474385,0.241108,0.466911,0.434065
7,0.467600,0.478042,0.163872,0.477605,0.239284,0.468635,0.437644
8,0.434900,0.476956,0.187689,0.494259,0.258788,0.483407,0.456956
9,0.435100,0.475347,0.188789,0.494190,0.262514,0.485608,0.459629
10,0.416600,0.475664,0.189321,0.492966,0.258012,0.482432,0.461262


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


{'eval_loss': 0.47229379415512085, 'eval_bleu': 0.19156588038375424, 'eval_rouge1': 0.49821472572638525, 'eval_rouge2': 0.26037547294190294, 'eval_rougeL': 0.48608466436493974, 'eval_meteor': 0.4626056059854938, 'eval_runtime': 1.979, 'eval_samples_per_second': 30.319, 'eval_steps_per_second': 4.042, 'epoch': 14.0}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 71615.89it/s]

Final predictions with inputs saved to predicted_clarif_T5SMALL_TOP10DOC_TFIDF.jsonl


eval/bleu,▁▃▃▄▅▅▅▇▇▇█▇▇██
eval/loss,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁
eval/meteor,▁▄▄▅▆▆▇████████
eval/rouge1,▁▄▄▆▆▆▆▇▇▇█████
eval/rouge2,▁▂▃▄▄▆▅▇█▇█▇███
eval/rougeL,▁▄▅▆▆▇▇████████
eval/runtime,▇███▃▃▂▁▂▁▃▂▂▄▄
eval/samples_per_second,▂▁▁▁▆▆▇█▇█▆▇▇▅▅
eval/steps_per_second,▂▁▁▁▆▆▆█▇█▆▇▇▅▅
test/bleu,▁
test/loss,▁


# TOP 10 DOCUMENTS, T5 SMALL, TDIFD CLEANED !!

In [30]:

tokenizer,tokenized_train_dataset, tokenized_eval_dataset= tokenize_and_split_dataset("training_top10_qulac_PREPROCESSED_FOR_MODEL_TFIDF_CLEANED.json","t5-small")
print(tokenized_train_dataset[0])  # Optional: check one example

Map:   0%|          | 0/539 [00:00<?, ? examples/s]

/users/Etu0/21402600/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/60 [00:00<?, ? examples/s]

{'input': "[QUERY] I'm looking for information on Rick Warren, the evangelical minister.\n[DOCUMENTS]\n[DOC 1] obama warren inauguration rick barack wright prayer jeremiah dahlia alexander evangelical gay post church ctl hanna inaugural rachael stroller political\n[DOC 2] warren obama pmreply bil lgbt bilerico rick comment december hate ve big inauguration community don gay waymon prayer whiny right\n[DOC 3] obama warren reply login rick register progressive barack liberal fundie angelfoodmcspade invocation thu gay heart folk ceremony evangelical youcanthandledatruth ohmmade\n[DOC 4] warren ucc rick inauguration obama church rev currie wrong blog voice progressive christ portland chuck parkrose religious faith opposed oregon\n[DOC 5] et obama pm warren evangelical rick gay invocation people barack god president choice bible elect mccain think right abortion religious\n[DOC 6] obama gay warren dec flag reply marriage civil japhy evangelical post roosevelt right washington white str invi

In [32]:
wandb.init(
    project="T5-Clarifications",  
    name="T5 SMALL + TOP 10 DOCS TFIDF CLEANED "                   
)
train("t5-small", "predicted_clarif_T5SMALL_TOP10DOC_TFIDF_CLEANED.jsonl",tokenizer, tokenized_train_dataset, tokenized_eval_dataset,40)


All TF 2.0 model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_1389011/3298686746.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor
1,20.694700,4.167738,0.024244,0.371587,0.191891,0.364078,0.222026
2,20.694700,0.811859,0.107319,0.357107,0.168929,0.349676,0.286539
3,4.591700,0.742543,0.128234,0.391908,0.192670,0.383771,0.330736
4,4.591700,0.689369,0.135222,0.403324,0.195763,0.390824,0.349692
5,4.591700,0.642927,0.132013,0.404858,0.201291,0.392272,0.354776
6,0.807600,0.573485,0.161868,0.441045,0.229638,0.428007,0.390647
7,0.807600,0.540802,0.157325,0.442252,0.226629,0.434822,0.396413
8,0.807600,0.520051,0.167385,0.460135,0.234141,0.450140,0.411276
9,0.587700,0.510913,0.161534,0.463689,0.238924,0.453761,0.420154
10,0.587700,0.505721,0.167697,0.469235,0.242106,0.460769,0.423740


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

{'eval_loss': 0.47561225295066833, 'eval_bleu': 0.19599575082949333, 'eval_rouge1': 0.4952546724913716, 'eval_rouge2': 0.2684139504411648, 'eval_rougeL': 0.4851800011878169, 'eval_meteor': 0.45739784690501534, 'eval_runtime': 0.6696, 'eval_samples_per_second': 89.609, 'eval_steps_per_second': 2.987, 'epoch': 40.0}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 72712.58it/s]

Final predictions with inputs saved to predicted_clarif_T5SMALL_TOP10DOC_TFIDF_CLEANED.jsonl


eval/bleu,▁▄▅▆▅▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇████████████████████
eval/loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/meteor,▁▃▄▅▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇█████████████████████
eval/rouge1,▂▁▃▃▃▅▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇██████████████████
eval/rouge2,▃▁▃▃▃▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇██████████████████
eval/rougeL,▂▁▃▃▃▅▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇██████████████████
eval/runtime,▂▂▂▂▁▂▂▂▂▂▂█▂▂▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▁
eval/samples_per_second,▆▇▇▇▇▇▇▇▇▇▇▁▇▇▇▇▇▇▇▇▆▇▇▇▇▆▇▇▇▇▇▇▇▆▇▇▇▇▇█
eval/steps_per_second,▆▇▇▇▇▇▇▇▇▇▇▁▇▇▇▇▇▇▇▇▆▇▇▇▇▆▇▇▇▇▇▇▇▆▇▇▇▇▇█
test/bleu,▁
test/loss,▁


In [33]:
# TFIFDF CLEANED TOP 15 WITH T5SMALL

In [44]:

tokenizer,tokenized_train_dataset, tokenized_eval_dataset= tokenize_and_split_dataset("training_top5_qulac_PREPROCESSED_FOR_MODEL_TFIDF_CLEANED.json","t5-small")
print(tokenized_train_dataset[0])  # Optional: check one example

Map:   0%|          | 0/539 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

{'input': "[QUERY] I'm looking for information on Rick Warren, the evangelical minister.\n[DOCUMENTS]\n[DOC 1] obama warren inauguration rick barack wright prayer jeremiah dahlia alexander evangelical gay post church ctl hanna inaugural rachael stroller political\n[DOC 2] warren obama pmreply bil lgbt bilerico rick comment december hate ve big inauguration community don gay waymon prayer whiny right\n[DOC 3] obama warren reply login rick register progressive barack liberal fundie angelfoodmcspade invocation thu gay heart folk ceremony evangelical youcanthandledatruth ohmmade\n[DOC 4] warren ucc rick inauguration obama church rev currie wrong blog voice progressive christ portland chuck parkrose religious faith opposed oregon\n[DOC 5] et obama pm warren evangelical rick gay invocation people barack god president choice bible elect mccain think right abortion religious", 'output': "I'm looking for background and biographical information on Rick Warren.", 'input_ids': [784, 15367, 11824, 

In [45]:
wandb.init(
    project="T5-Clarifications",  
    name="T5 SMALL + TOP5 DOCS TFIDF CLEANED "                   
)
train("t5-small", "predicted_clarif_T5SMALL_TOP5DOC_TFIDF_CLEANED.jsonl",tokenizer, tokenized_train_dataset, tokenized_eval_dataset,40)


All TF 2.0 model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_1389011/287362180.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor
1,6.020100,0.750591,0.111133,0.382052,0.180609,0.370262,0.314686
2,0.998600,0.590119,0.144853,0.425262,0.210200,0.417045,0.370997
3,0.612400,0.521921,0.167089,0.457482,0.234683,0.446116,0.406463
4,0.540400,0.503287,0.169154,0.456124,0.229099,0.448066,0.409783
5,0.511800,0.497196,0.164821,0.464146,0.226652,0.455479,0.415864
6,0.470900,0.488893,0.157548,0.463477,0.226462,0.456482,0.416965
7,0.476000,0.485329,0.161877,0.470216,0.234615,0.460682,0.427468
8,0.438600,0.482569,0.167427,0.471517,0.231280,0.463674,0.425976
9,0.442000,0.480201,0.180948,0.479859,0.243026,0.471497,0.436291
10,0.419600,0.478255,0.177844,0.478104,0.237394,0.468293,0.436926


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


{'eval_loss': 0.4771442413330078, 'eval_bleu': 0.17996959865803996, 'eval_rouge1': 0.4817831447099876, 'eval_rouge2': 0.24131369566392785, 'eval_rougeL': 0.4699829025534148, 'eval_meteor': 0.4395113427286545, 'eval_runtime': 2.0622, 'eval_samples_per_second': 29.095, 'eval_steps_per_second': 3.879, 'epoch': 17.0}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 97504.16it/s]

Final predictions with inputs saved to predicted_clarif_T5SMALL_TOP5DOC_TFIDF_CLEANED.jsonl


eval/bleu,▁▄▆▆▆▅▆▆▇▇▇▇▇▇▇▇█▇
eval/loss,█▄▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/meteor,▁▄▆▆▆▆▇▇██████████
eval/rouge1,▁▄▆▆▆▆▇▇▇▇█████▇██
eval/rouge2,▁▄▇▆▆▆▇▇█▇█▇▇█▇▇██
eval/rougeL,▁▄▆▆▇▇▇▇██████████
eval/runtime,▆▄▄▂▁▂▃▄▃▃▆▃▃▃▅▅▆█
eval/samples_per_second,▃▅▅▇█▇▆▅▆▆▃▅▆▆▄▄▃▁
eval/steps_per_second,▃▅▅▇█▇▆▅▆▆▃▅▆▆▄▄▃▁
test/bleu,▁
test/loss,▁


# -----------------------------------------------------------------

# Cross validation

# T5 EFFICIENT -------------- TOP5 RAW DOCS 

In [4]:
from sklearn.model_selection import KFold
from datasets import Dataset
from transformers import AutoTokenizer
import json
import numpy as np

#fucntion to tokenize the dataset based on the model
def tokenize_and_split_dataset(filename, modelname):
    with open(filename, "r", encoding="utf-8") as f:
        raw_data = json.load(f)
        
    # Choose your model
    tokenizer = AutoTokenizer.from_pretrained(modelname)

    # Tokenization function
    def preprocess(batch):
        model_inputs = tokenizer(
            batch["input"],
            max_length=512,
            padding="max_length",
            truncation=True
        )
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(
                batch["output"],
                max_length=64,
                padding="max_length",
                truncation=True
            )
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs
    
    # Convert raw data to Dataset object
    dataset = Dataset.from_list(raw_data)
    tokenized_dataset = dataset.map(preprocess, batched=True)

    return tokenizer, tokenized_dataset


In [5]:
from sklearn.model_selection import KFold

#fucntion to split dataset into folds
def generate_folds(tokenized_dataset, n_splits=5, seed=42):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    folds = list(kf.split(tokenized_dataset))
    return folds


In [6]:
import wandb
from transformers import AutoModelForSeq2SeqLM, Trainer, TrainingArguments, EarlyStoppingCallback, TrainerCallback
from datasets import Dataset
import numpy as np
import json
import os
from tqdm import tqdm

# Custom callback to log metrics after each evaluation step
class WandbLoggingCallback(TrainerCallback):
    def __init__(self, fold_num):
        self.fold_num = fold_num

    def on_evaluate(self, args, state, control, metrics, **kwargs):
        if state.is_world_process_zero:
            log_data = {f"Fold_{self.fold_num}/eval/{key}": value for key, value in metrics.items()}
            wandb.log(log_data)

def cross_val_train(modelname, filename, run_name, tokenizer, tokenized_dataset, folds, nb_epochs=30):
    # Start a single WandB run for all folds
    wandb.init(project="cross_val_T5", name=run_name, config={"epochs": nb_epochs})
    filenamebis= filename.split(".jsonl")[0]
    fold_metrics = []

    for fold_num, (train_idx, test_idx) in enumerate(folds):
        print(f"Processing Fold {fold_num + 1}")

        # Split the dataset
        train_dataset = tokenized_dataset.select(train_idx)
        test_dataset = tokenized_dataset.select(test_idx)

        # Initialize the model
        model = AutoModelForSeq2SeqLM.from_pretrained(modelname, from_tf=True)

        # Training arguments with early stopping
        training_args = TrainingArguments(
            output_dir=f"./results/{run_name}/fold_{fold_num+1}",
            eval_strategy="epoch",
            save_strategy="epoch",
            learning_rate=5e-5,
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            num_train_epochs=nb_epochs,
            load_best_model_at_end=True,
            logging_steps=20,
            logging_first_step=True,
            metric_for_best_model="eval_loss",
            greater_is_better=False,
            seed=42,
        )

        # Initialize Trainer with the logging callback and early stopping
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=test_dataset,
            tokenizer=tokenizer,
            # compute_metrics=compute_metrics,
            compute_metrics=lambda eval_pred: compute_metrics(eval_pred, tokenizer),
            callbacks=[
                WandbLoggingCallback(fold_num + 1),
                EarlyStoppingCallback(early_stopping_patience=3)
            ],
        )

        # Train the model
        trainer.train()

        # Final evaluation for the fold
        eval_results = trainer.evaluate()
        print(f"Fold {fold_num + 1} results: {eval_results}")

        # Store the final evaluation results for this fold
        fold_metrics.append(eval_results)

        # Predict on the test dataset
        predictions = trainer.predict(test_dataset)

    
        # Decode predictions and save to file
        pred_ids = np.argmax(predictions.predictions[0], axis=-1)
        predicted_texts = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
        true_texts = tokenizer.batch_decode(predictions.label_ids, skip_special_tokens=True)
        input_ids = test_dataset["input_ids"]
        input_texts = tokenizer.batch_decode(input_ids, skip_special_tokens=True)

        # Save predictions to a file

        output_predictions_file = f"{filenamebis}_fold_{fold_num+1}.jsonl"
        with open(output_predictions_file, "w", encoding="utf-8") as writer:
            for input_text, true_text, predicted_text in tqdm(zip(input_texts, true_texts, predicted_texts), total=len(true_texts)):
                output_dict = {
                    "input": input_text,
                    "true": true_text,
                    "predicted": predicted_text
                }
                writer.write(json.dumps(output_dict, ensure_ascii=False) + "\n")
        
        print(f"Predictions for fold {fold_num + 1} saved to {output_predictions_file}")

    # Calculate and log average metrics
    avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics]) for key in fold_metrics[0]}
    wandb.log({f"Averages/{key}": value for key, value in avg_metrics.items()})
    print("Average metrics over all folds:", avg_metrics)

    wandb.finish()

    #merge all to a single file for predcitions

    with open(filename, "w", encoding="utf-8") as merged_file:
        for fold_num in range(len(folds)):
            fold_file = f"{filenamebis}_fold_{fold_num+1}.jsonl"
            if os.path.exists(fold_file):
                with open(fold_file, "r", encoding="utf-8") as f:
                    for line in f:
                        merged_file.write(line)
    print(f"Final predicitons for all folds foudn in {filename}")


In [47]:
tokenizer,tokenized_dataset= tokenize_and_split_dataset("training_top5_qulac_PREPROCESSED_FOR_MODEL.json","google/t5-efficient-tiny")
print(tokenized_dataset[0])  # Optional: check one example

Map:   0%|          | 0/599 [00:00<?, ? examples/s]

/users/Etu0/21402600/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input': '[QUERY] Find information on President Barack Obama\'s family history, including genealogy, national origins, places and dates of birth, etc.\n[DOCUMENTS]\n[DOC 1] Family of Barack Obama - Wikipedia, the free encyclopedia\\nFamily of Barack Obama\\nFrom Wikipedia, the free encyclopedia\\n(Redirected from Sasha Obama)\\nJump to: navigation, search\\nObama Family\\nPresident Barack Obama, First Lady Michelle, and daughters Malia and Sasha wave to the crowd after his inaugural address Jan. 20, 2009, on the west steps of the U.S. Capitol.[1]\\nCurrent region\\nWashington, DC\\nInformation\\nPlace of origin\\nUnited States\\nNotable members\\nBarack Obama, Michelle Obama, Ann Dunham, Barack Obama, Sr., etc.\\nConnected families\\nRobinson, Dunham, Soetoro, Ng\\nThis article is part of a series about\\nBarack Obama\\nBackground \xa0ýý Illinois Senate \xa0ýý U.S. Senate\\nPolitical positions\xa0ýý Public image\xa0ýý Family\\n2008 primaries\xa0ýý ObamaýýýBiden campaign\\nTransition\x

In [48]:
folds= generate_folds(tokenized_dataset, n_splits=5)

In [50]:
cross_val_train(modelname="google/t5-efficient-tiny", run_name="T5_Tiny_TOP_5_DOCS",filename="crossval_T5_TINY_TOP5DOCS.jsonl", 
                tokenizer=tokenizer, tokenized_dataset=tokenized_dataset, 
                folds=folds, nb_epochs=30)

Processing Fold 1


All TF 2.0 model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_1981337/3761292519.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor
1,19.895900,12.165541,0.000000,0.041692,0.006561,0.041580,0.017722
2,16.623700,7.866652,0.010369,0.157073,0.034147,0.155178,0.077415
3,11.413700,4.419832,0.016602,0.205159,0.028687,0.198033,0.099604
4,7.553300,1.940492,0.016824,0.227160,0.029911,0.223101,0.125728
5,7.553300,1.239293,0.000000,0.212471,0.025821,0.206655,0.115816
6,4.606600,1.288048,0.017448,0.208856,0.032854,0.206730,0.112172
7,2.826900,1.344526,0.000000,0.202133,0.026542,0.194540,0.119747
8,1.909600,1.305108,0.000000,0.195121,0.024606,0.191752,0.113181


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

Fold 1 results: {'eval_loss': 1.2392934560775757, 'eval_bleu': 0.0, 'eval_rouge1': 0.21247113604066634, 'eval_rouge2': 0.025821114272201226, 'eval_rougeL': 0.2066552314766033, 'eval_meteor': 0.11581556071151769, 'eval_runtime': 1.3604, 'eval_samples_per_second': 88.208, 'eval_steps_per_second': 2.94, 'epoch': 8.0}


...

Fold 5 results: {'eval_loss': 1.1881837844848633, 'eval_bleu': 0.0, 'eval_rouge1': 0.2440319997795114, 'eval_rouge2': 0.03721688181648636, 'eval_rougeL': 0.2328550349254946, 'eval_meteor': 0.14662765593001656, 'eval_runtime': 1.3179, 'eval_samples_per_second': 90.297, 'eval_steps_per_second': 3.035, 'epoch': 8.0}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119/119 [00:00<00:00, 46879.14it/s]

Predictions for fold 5 saved to crossval_T5_TINY_TOP5DOCS_fold_5.jsonl
Average metrics over all folds: {'eval_loss': 1.2078011512756348, 'eval_bleu': 0.0026892462852327265, 'eval_rouge1': 0.2265923110136563, 'eval_rouge2': 0.030546592997341603, 'eval_rougeL': 0.21755901095964383, 'eval_meteor': 0.13422780003592608, 'eval_runtime': 1.35178, 'eval_samples_per_second': 88.6914, 'eval_steps_per_second': 2.9612000000000003, 'epoch': 8.0}


Averages/epoch,▁
Averages/eval_bleu,▁
Averages/eval_loss,▁
Averages/eval_meteor,▁
Averages/eval_rouge1,▁
Averages/eval_rouge2,▁
Averages/eval_rougeL,▁
Averages/eval_runtime,▁
Averages/eval_samples_per_second,▁
Averages/eval_steps_per_second,▁
Fold_1/eval/epoch,▁▂▃▄▅▆▇██


Final predicitons for all folds foudn in crossval_T5_TINY_TOP5DOCS.jsonl


In [ ]:
#WORKING VERSION OF CV WITHOUT PROPER LOGGING

# import wandb
# from transformers import AutoModelForSeq2SeqLM, Trainer, TrainingArguments, EarlyStoppingCallback
# from datasets import Dataset
# import os
# from tqdm import tqdm
# from sklearn.model_selection import KFold
# import numpy as np

# def cross_val_train(modelname, filename, run_name, tokenizer, tokenized_dataset, folds, nb_epochs=30):
#     # Start a single WandB run for all folds
#     wandb.init(project="cross_val_Test", name=run_name, config={"epochs": nb_epochs})

#     fold_metrics = []

#     for fold_num, (train_idx, test_idx) in enumerate(folds):
#         print(f"Processing Fold {fold_num + 1}")

#         # Split the dataset
#         train_dataset = tokenized_dataset.select(train_idx)
#         test_dataset = tokenized_dataset.select(test_idx)

#         # Initialize the model for each fold
#         model = AutoModelForSeq2SeqLM.from_pretrained(modelname, from_tf=True)

#         # Training arguments
#         training_args = TrainingArguments(
#             output_dir=f"./results/fold_{fold_num+1}",  # Different folder per fold
#             eval_strategy="epoch",
#             save_strategy="epoch",
#             learning_rate=5e-5,
#             per_device_train_batch_size=8,
#             per_device_eval_batch_size=8,
#             num_train_epochs=nb_epochs,
#             logging_dir=f"./results/logs/fold_{fold_num+1}",
#             load_best_model_at_end=True,
#             metric_for_best_model="eval_loss",
#             greater_is_better=False,
#             seed=42,  # Seed for reproducibility
#         )

#         # Initialize Trainer
#         trainer = Trainer(
#             model=model,
#             args=training_args,
#             train_dataset=train_dataset,
#             eval_dataset=test_dataset,
#             tokenizer=tokenizer,
#             compute_metrics=compute_metrics,
#         )

#         # Train the model
#         trainer.train()

#         # Evaluate the model on this fold
#         eval_results = trainer.evaluate()
#         print(f"Fold {fold_num + 1} results: {eval_results}")

#         # Store the fold metrics
#         fold_metrics.append(eval_results)

#     # Aggregate results over all folds
#     avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics]) for key in fold_metrics[0]}
#     print("Average metrics over all folds:", avg_metrics)

#     # Log average metrics to WandB

#     # End the WandB run
#     wandb.finish()


In [6]:
# import wandb
# from transformers import AutoModelForSeq2SeqLM, Trainer, TrainingArguments, EarlyStoppingCallback, TrainerCallback
# from datasets import Dataset
# import numpy as np

# # Custom callback to log metrics after each evaluation step
# class WandbLoggingCallback(TrainerCallback):
#     def __init__(self, fold_num):
#         self.fold_num = fold_num

#     def on_evaluate(self, args, state, control, metrics, **kwargs):
#         if state.is_world_process_zero:
#             # Log metrics with fold-specific keys
#             log_data = {f"Fold_{self.fold_num}/eval/{key}": value for key, value in metrics.items()}
#             wandb.log(log_data)

# def cross_val_train(modelname, filename, run_name, tokenizer, tokenized_dataset, folds, nb_epochs=30):
#     # Start a single WandB run for all folds
#     wandb.init(project="cross_val_T5", name=run_name, config={"epochs": nb_epochs})

#     fold_metrics = []

#     for fold_num, (train_idx, test_idx) in enumerate(folds):
#         print(f"Processing Fold {fold_num + 1}")

#         # Split the dataset
#         train_dataset = tokenized_dataset.select(train_idx)
#         test_dataset = tokenized_dataset.select(test_idx)

#         # Initialize the model
#         model = AutoModelForSeq2SeqLM.from_pretrained(modelname, from_tf=True)

#         # Training arguments with early stopping
#         training_args = TrainingArguments(
#             output_dir=f"./results/fold_{fold_num+1}",
#             eval_strategy="epoch",
#             save_strategy="epoch",
#             learning_rate=5e-5,
#             per_device_train_batch_size=8,
#             per_device_eval_batch_size=8,
#             num_train_epochs=nb_epochs,
#             logging_dir=f"./results/logs/fold_{fold_num+1}",
#             load_best_model_at_end=True,
#             logging_steps=20,
#             logging_first_step=True,
#             metric_for_best_model="eval_loss",
#             greater_is_better=False,
#             seed=42,
#         )

#         # Initialize Trainer with the logging callback and early stopping
#         trainer = Trainer(
#             model=model,
#             args=training_args,
#             train_dataset=train_dataset,
#             eval_dataset=test_dataset,
#             tokenizer=tokenizer,
#             compute_metrics=compute_metrics,
#             callbacks=[
#                 WandbLoggingCallback(fold_num + 1),  # Log metrics for each fold
#                 EarlyStoppingCallback(early_stopping_patience=3)  # Early stopping after 3 epochs of no improvement
#             ],
#         )

#         # Train the model
#         trainer.train()

#         # Final evaluation for the fold
#         eval_results = trainer.evaluate()
#         print(f"Fold {fold_num + 1} results: {eval_results}")

#         # Store the final evaluation results for this fold
#         fold_metrics.append(eval_results)

#     # Calculate and log average metrics
#     avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics]) for key in fold_metrics[0]}
#     wandb.log({f"Averages/{key}": value for key, value in avg_metrics.items()})
#     print("Average metrics over all folds:", avg_metrics)

#     wandb.finish()


# T5 SMALL ------------------- TO5 RAW DOCS

In [51]:
tokenizer,tokenized_dataset= tokenize_and_split_dataset("training_top5_qulac_PREPROCESSED_FOR_MODEL.json","t5-small")
print(tokenized_dataset[0])  # Optional: check one example

Map:   0%|          | 0/599 [00:00<?, ? examples/s]

/users/Etu0/21402600/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input': '[QUERY] Find information on President Barack Obama\'s family history, including genealogy, national origins, places and dates of birth, etc.\n[DOCUMENTS]\n[DOC 1] Family of Barack Obama - Wikipedia, the free encyclopedia\\nFamily of Barack Obama\\nFrom Wikipedia, the free encyclopedia\\n(Redirected from Sasha Obama)\\nJump to: navigation, search\\nObama Family\\nPresident Barack Obama, First Lady Michelle, and daughters Malia and Sasha wave to the crowd after his inaugural address Jan. 20, 2009, on the west steps of the U.S. Capitol.[1]\\nCurrent region\\nWashington, DC\\nInformation\\nPlace of origin\\nUnited States\\nNotable members\\nBarack Obama, Michelle Obama, Ann Dunham, Barack Obama, Sr., etc.\\nConnected families\\nRobinson, Dunham, Soetoro, Ng\\nThis article is part of a series about\\nBarack Obama\\nBackground \xa0ýý Illinois Senate \xa0ýý U.S. Senate\\nPolitical positions\xa0ýý Public image\xa0ýý Family\\n2008 primaries\xa0ýý ObamaýýýBiden campaign\\nTransition\x

In [ ]:
folds= generate_folds(tokenized_dataset, n_splits=5)

In [53]:
cross_val_train(modelname="t5-small", run_name="T5_SMALL_TOP_5_DOCS",filename="crossval_T5_SMALL_TOP5DOCS.jsonl", 
                tokenizer=tokenizer, tokenized_dataset=tokenized_dataset, 
                folds=folds, nb_epochs=30)

Processing Fold 1


All TF 2.0 model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_1981337/3761292519.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor
1,24.110600,7.909437,0.047588,0.374766,0.157747,0.363523,0.275237
2,12.190700,1.011101,0.103273,0.386857,0.166034,0.377508,0.301269
3,2.538600,0.872277,0.114142,0.398919,0.178434,0.387180,0.319090
4,1.164700,0.828989,0.117945,0.409622,0.173707,0.393014,0.330334
5,1.164700,0.721032,0.135739,0.427961,0.195236,0.414530,0.358241
6,0.847300,0.673097,0.143557,0.433076,0.206606,0.422220,0.369737
7,0.747500,0.636529,0.148427,0.436883,0.212886,0.428279,0.378551
8,0.657900,0.590409,0.159183,0.450980,0.225189,0.442551,0.391678
9,0.657900,0.556572,0.167652,0.474435,0.229413,0.466709,0.418650
10,0.583200,0.544981,0.171319,0.479601,0.226465,0.472042,0.423932


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

Fold 1 results: {'eval_loss': 0.5113382935523987, 'eval_bleu': 0.1893099536860164, 'eval_rouge1': 0.4949540176339151, 'eval_rouge2': 0.24489164019572845, 'eval_rougeL': 0.4857354966359395, 'eval_meteor': 0.44466199798519823, 'eval_runtime': 1.6267, 'eval_samples_per_second': 73.77, 'eval_steps_per_second': 2.459, 'epoch': 30.0}


...

Fold 5 results: {'eval_loss': 0.47231248021125793, 'eval_bleu': 0.205260004896688, 'eval_rouge1': 0.49837764631903186, 'eval_rouge2': 0.23523881251568848, 'eval_rougeL': 0.4885402364015746, 'eval_meteor': 0.45092694757338664, 'eval_runtime': 1.7126, 'eval_samples_per_second': 69.486, 'eval_steps_per_second': 2.336, 'epoch': 30.0}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119/119 [00:00<00:00, 44648.20it/s]

Predictions for fold 5 saved to crossval_T5_SMALL_TOP5DOCS_fold_5.jsonl
Average metrics over all folds: {'eval_loss': 0.48240370750427247, 'eval_bleu': 0.19105834210343983, 'eval_rouge1': 0.48841880395297943, 'eval_rouge2': 0.2405301203580545, 'eval_rougeL': 0.48020857517432863, 'eval_meteor': 0.44378319799755195, 'eval_runtime': 1.8285399999999998, 'eval_samples_per_second': 66.708, 'eval_steps_per_second': 2.2276, 'epoch': 30.0}


Averages/epoch,▁
Averages/eval_bleu,▁
Averages/eval_loss,▁
Averages/eval_meteor,▁
Averages/eval_rouge1,▁
Averages/eval_rouge2,▁
Averages/eval_rougeL,▁
Averages/eval_runtime,▁
Averages/eval_samples_per_second,▁
Averages/eval_steps_per_second,▁
Fold_1/eval/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇████


Final predicitons for all folds foudn in crossval_T5_SMALL_TOP5DOCS.jsonl


# T5 SMALL----------------- TOP5 QUERY ONLY

In [9]:
tokenizer,tokenized_dataset= tokenize_and_split_dataset("training_queryonly_qulac_PREPROCESSED_FOR_MODEL.json","t5-small")
print(tokenized_dataset[0])  # Optional: check one example

Map:   0%|          | 0/599 [00:00<?, ? examples/s]

{'input': "Find information on President Barack Obama's family history, including genealogy, national origins, places and dates of birth, etc.", 'output': "Where did Barack Obama's parents and grandparents come from?", 'input_ids': [2588, 251, 30, 1661, 20653, 4534, 31, 7, 384, 892, 6, 379, 31960, 6, 1157, 5233, 7, 6, 1747, 11, 5128, 13, 3879, 6, 672, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

/users/Etu0/21402600/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [10]:
folds= generate_folds(tokenized_dataset, n_splits=5)

In [11]:
cross_val_train(modelname="t5-small", run_name="T5_SMALL_QUERY_ONLY",filename="crossval_T5_SMALL_QUERY_ONLY.jsonl", 
                tokenizer=tokenizer, tokenized_dataset=tokenized_dataset, 
                folds=folds, nb_epochs=30)

Processing Fold 1


W0000 00:00:1746893712.776653    2863 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
All TF 2.0 model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_2863/3761292519.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the

Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor
1,1.855500,0.994875,0.098887,0.342733,0.148488,0.331003,0.283723
2,0.923700,0.791439,0.126398,0.393167,0.187927,0.382688,0.332641
3,0.734600,0.619414,0.150396,0.431079,0.207093,0.418767,0.377791
4,0.591800,0.566767,0.160119,0.457525,0.212743,0.444881,0.407786
5,0.556700,0.550978,0.167941,0.462883,0.221320,0.449165,0.412682
6,0.518600,0.541416,0.176522,0.472386,0.229268,0.457819,0.424240
7,0.493400,0.533020,0.188335,0.480884,0.239085,0.466859,0.437689
8,0.493900,0.527780,0.192626,0.478057,0.239658,0.465535,0.435083
9,0.492900,0.523187,0.190525,0.486239,0.242472,0.468519,0.439281
10,0.468100,0.519190,0.187495,0.482558,0.240865,0.465692,0.437932


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


Fold 1 results: {'eval_loss': 0.5081183910369873, 'eval_bleu': 0.19618116080673445, 'eval_rouge1': 0.48946004794296377, 'eval_rouge2': 0.24238264179214067, 'eval_rougeL': 0.4718249742908727, 'eval_meteor': 0.4396262780902425, 'eval_runtime': 2.2058, 'eval_samples_per_second': 54.402, 'eval_steps_per_second': 6.8, 'epoch': 26.0}


...

Fold 5 results: {'eval_loss': 0.4666701555252075, 'eval_bleu': 0.2071235891410905, 'eval_rouge1': 0.49703457415389246, 'eval_rouge2': 0.23749999177333164, 'eval_rougeL': 0.48718710276889843, 'eval_meteor': 0.4508434373130092, 'eval_runtime': 2.4394, 'eval_samples_per_second': 48.782, 'eval_steps_per_second': 6.149, 'epoch': 30.0}


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119/119 [00:00<00:00, 112465.56it/s]

Predictions for fold 5 saved to crossval_T5_SMALL_QUERY_ONLY_fold_5.jsonl
Average metrics over all folds: {'eval_loss': 0.47526371479034424, 'eval_bleu': 0.19411330054290937, 'eval_rouge1': 0.4892978206261313, 'eval_rouge2': 0.24191867763495162, 'eval_rougeL': 0.4783860218231945, 'eval_meteor': 0.4459745481087546, 'eval_runtime': 2.2956600000000003, 'eval_samples_per_second': 52.59060000000001, 'eval_steps_per_second': 6.5840000000000005, 'epoch': 26.6}


Averages/epoch,▁
Averages/eval_bleu,▁
Averages/eval_loss,▁
Averages/eval_meteor,▁
Averages/eval_rouge1,▁
Averages/eval_rouge2,▁
Averages/eval_rougeL,▁
Averages/eval_runtime,▁
Averages/eval_samples_per_second,▁
Averages/eval_steps_per_second,▁
Fold_1/eval/epoch,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇███


Final predicitons for all folds foudn in crossval_T5_SMALL_QUERY_ONLY.jsonl


# ---------- TOP 5 DOUCMENTS SUMMARIZED

In [12]:
tokenizer,tokenized_dataset= tokenize_and_split_dataset("training_top5_qulac_SUMMARIZED_FOR_MODEL.json","t5-small")
print(tokenized_dataset[0])  # Optional: check one example

Map:   0%|          | 0/599 [00:00<?, ? examples/s]

{'input': "[QUERY] Find information on President Barack Obama's family history, including genealogy, national origins, places and dates of birth, etc.\n[DOCUMENTS]\n[DOC 1] This page was last modified on 14 March 2009, at 05:50 . It contains articles with unsourced statements and articles needing factual verification since October 2008 .\n[DOC 2] This page was last modified on 14 March 2009, at 05:50 . It contains articles needing factual verification since October 2008 . It is available under the terms of the GNU Free Documentation License .\n[DOC 3] This page was last modified on 16 March 2009, at 03:22 . It contains articles with unsourced statements and articles needing factual verification since October 2008 . Hidden categories: Wikipedia semi-protected pages | All pages needing cleanup .\n[DOC 4] This page was last modified on 14 March 2009, at 05:50 . Use this page to help users with reading comprehension and vocabulary . Use the weekly Newsquiz to test your knowledge of stories

In [13]:
folds= generate_folds(tokenized_dataset, n_splits=5)

In [14]:
cross_val_train(modelname="t5-small", run_name="T5_SMALL_TOP5DOCS_SUMMARIZED",filename="crossval_T5_TOP5DOCS_SUMMARIZED.jsonl", 
                tokenizer=tokenizer, tokenized_dataset=tokenized_dataset, 
                folds=folds, nb_epochs=30)

Processing Fold 1


All TF 2.0 model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_14586/3145572364.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor
1,1.393400,0.844550,0.098140,0.393720,0.172560,0.383675,0.300530
2,0.730600,0.601212,0.150760,0.431580,0.206855,0.424689,0.377058
3,0.586500,0.545290,0.170607,0.469422,0.220446,0.459851,0.417890
4,0.519100,0.530222,0.178776,0.470718,0.228101,0.461492,0.423201
5,0.491400,0.515488,0.182384,0.477691,0.228463,0.468648,0.429876
6,0.460800,0.508271,0.182371,0.478430,0.229796,0.469196,0.429252
7,0.438900,0.503539,0.185633,0.474441,0.231480,0.463858,0.425513
8,0.429800,0.499806,0.180491,0.475617,0.225040,0.467359,0.426904
9,0.432800,0.499436,0.186712,0.484756,0.230496,0.472845,0.436418
10,0.413600,0.499159,0.193403,0.480349,0.237614,0.470217,0.433564


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


Fold 1 results: {'eval_loss': 0.4986312985420227, 'eval_bleu': 0.19074583115428184, 'eval_rouge1': 0.48904300069121504, 'eval_rouge2': 0.23124195063454211, 'eval_rougeL': 0.4744587812246184, 'eval_meteor': 0.43838852433833125, 'eval_runtime': 2.1723, 'eval_samples_per_second': 55.242, 'eval_steps_per_second': 6.905, 'epoch': 15.0}


...

Fold 5 results: {'eval_loss': 0.45843765139579773, 'eval_bleu': 0.20557456979612207, 'eval_rouge1': 0.5166019243725455, 'eval_rouge2': 0.25090219860397556, 'eval_rougeL': 0.5088385907134892, 'eval_meteor': 0.4691332159650356, 'eval_runtime': 2.0821, 'eval_samples_per_second': 57.155, 'eval_steps_per_second': 7.204, 'epoch': 17.0}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119/119 [00:00<00:00, 41414.05it/s]

Predictions for fold 5 saved to crossval_T5_TOP5DOCS_SUMMARIZED_fold_5.jsonl
Average metrics over all folds: {'eval_loss': 0.4672218382358551, 'eval_bleu': 0.19918105626307475, 'eval_rouge1': 0.4996960822925914, 'eval_rouge2': 0.251680731174653, 'eval_rougeL': 0.4900307651693466, 'eval_meteor': 0.45689439464484566, 'eval_runtime': 2.12884, 'eval_samples_per_second': 56.2888, 'eval_steps_per_second': 7.048, 'epoch': 15.8}


Averages/epoch,▁
Averages/eval_bleu,▁
Averages/eval_loss,▁
Averages/eval_meteor,▁
Averages/eval_rouge1,▁
Averages/eval_rouge2,▁
Averages/eval_rougeL,▁
Averages/eval_runtime,▁
Averages/eval_samples_per_second,▁
Averages/eval_steps_per_second,▁
Fold_1/eval/epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇██


Final predicitons for all folds foudn in crossval_T5_TOP5DOCS_SUMMARIZED.jsonl


# HanDLE TFIDF WITH CROSS VAL

In [6]:
from sklearn.model_selection import KFold
from datasets import Dataset
from transformers import AutoTokenizer
import json
import numpy as np

#fucntion to tokenize the dataset based on the model
def tokenize_and_split_dataset(filename, modelname):
    with open(filename, "r", encoding="utf-8") as f:
        raw_data = json.load(f)
        
    # Choose your model
    tokenizer = AutoTokenizer.from_pretrained(modelname)

    # Tokenization function
    def preprocess(batch):
        model_inputs = tokenizer(
            batch["input"],
            max_length=512,
            padding="max_length",
            truncation=True
        )
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(
                batch["output"],
                max_length=64,
                padding="max_length",
                truncation=True
            )
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs
    
    # Convert raw data to Dataset object
    dataset = Dataset.from_list(raw_data)
    tokenized_dataset = dataset.map(preprocess, batched=True)

    return tokenizer, tokenized_dataset


In [7]:
import numpy as np
import json
def compute_metrics(eval_pred, tokenizer):
    """
    This function calculates BLEU, ROUGE, and METEOR for the model predictions.
    eval_pred: tuple (predictions, references), where predictions are model outputs
    and references are the true target labels.
    """
    predictions, labels = eval_pred
    predictions= predictions[0]
    pred_ids = np.argmax(predictions, axis=-1) 
    # Decode the model's predicted tokens and true labels
    predicted_texts = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    true_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Calculate BLEU score
    bleu_score = bleu.compute(predictions=predicted_texts, references=true_texts)
    rouge_score = rouge.compute(predictions=predicted_texts, references=true_texts)
    meteor_score = meteor.compute(predictions=predicted_texts, references=true_texts)
   
    # Combine the metrics into a dictionary
    return {
        "bleu": bleu_score["bleu"],
        "rouge1": rouge_score["rouge1"],
        "rouge2": rouge_score["rouge2"],
        "rougeL": rouge_score["rougeL"],
        "meteor": meteor_score["meteor"]
    }




In [8]:
from sklearn.model_selection import KFold

#fucntion to split dataset into folds
def generate_folds(tokenized_dataset, n_splits=5, seed=42):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    folds = list(kf.split(tokenized_dataset))
    return folds


In [9]:
# recuperer les folds pour les indices de chaque fichier tfidf
tokenizer_global,tokenized_dataset_global= tokenize_and_split_dataset("training_queryonly_qulac_PREPROCESSED_FOR_MODEL.json","t5-small")
folds= generate_folds(tokenized_dataset_global, n_splits=5)

Map:   0%|          | 0/599 [00:00<?, ? examples/s]

/users/Etu0/21402600/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [10]:
folds

[(array([  0,   1,   3,   4,   5,   7,   8,   9,  12,  13,  14,  15,  16,
          17,  18,  19,  20,  21,  22,  23,  25,  26,  27,  28,  31,  32,
          33,  34,  35,  36,  37,  38,  40,  41,  42,  43,  44,  45,  46,
          47,  48,  49,  50,  51,  52,  53,  54,  56,  57,  58,  59,  61,
          62,  64,  65,  66,  67,  68,  69,  71,  74,  75,  80,  84,  85,
          87,  88,  89,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100,
         102, 103, 105, 106, 107, 108, 111, 112, 113, 114, 115, 116, 117,
         119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
         133, 134, 136, 138, 139, 141, 142, 143, 144, 145, 146, 147, 149,
         150, 151, 152, 153, 154, 156, 157, 158, 159, 160, 161, 162, 164,
         166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 177, 178, 179,
         180, 183, 185, 186, 187, 188, 189, 190, 191, 193, 194, 196, 197,
         198, 200, 201, 202, 203, 204, 205, 206, 207, 211, 213, 214, 215,
         216, 217, 219, 220, 221, 222,

In [11]:
import wandb
from transformers import AutoModelForSeq2SeqLM, Trainer, TrainingArguments, EarlyStoppingCallback, TrainerCallback
from datasets import Dataset
import numpy as np
import json
import os
from tqdm import tqdm

# Custom callback to log metrics after each evaluation step
class WandbLoggingCallback(TrainerCallback):
    def __init__(self, fold_num):
        self.fold_num = fold_num

    def on_evaluate(self, args, state, control, metrics, **kwargs):
        if state.is_world_process_zero:
            log_data = {f"Fold_{self.fold_num}/eval/{key}": value for key, value in metrics.items()}
            wandb.log(log_data)

def cross_val_train(modelname, fold_filename, filename, run_name, tokenizer_global, tokenized_dataset_global, folds, nb_epochs=30):
    # Start a single WandB run for all folds
    wandb.init(project="cross_val_T5", name=run_name, config={"epochs": nb_epochs})
    filenamebis= filename.split(".jsonl")[0]
    fold_metrics = []

    for fold_num, (train_idx, test_idx) in enumerate(folds):
        print(f"Processing Fold {fold_num + 1}")
        
        #Ici on modifie pour recuperer le bon doc et appliquer le split dessus 3la 7ssab the fold
        file_tfdidf=fold_filename
        file_tfidf_fold=f"FOLD{fold_num}_{file_tfdidf}"
        tokenizer,tokenized_dataset= tokenize_and_split_dataset(file_tfidf_fold,modelname)
        print(f"Recuperation du Fichier {file_tfidf_fold} pour le training du Fold {fold_num}")
        
        # Split the dataset
        train_dataset = tokenized_dataset.select(train_idx)
        test_dataset = tokenized_dataset.select(test_idx)

        # Initialize the model
        model = AutoModelForSeq2SeqLM.from_pretrained(modelname, from_tf=True)

        # Training arguments with early stopping
        training_args = TrainingArguments(
            output_dir=f"./results/{run_name}/fold_{fold_num+1}",
            eval_strategy="epoch",
            save_strategy="epoch",
            learning_rate=5e-5,
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            num_train_epochs=nb_epochs,
            load_best_model_at_end=True,
            logging_steps=20,
            logging_first_step=True,
            metric_for_best_model="eval_loss",
            greater_is_better=False,
            seed=42,
        )

        # Initialize Trainer with the logging callback and early stopping
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=test_dataset,
            tokenizer=tokenizer,
            compute_metrics=lambda eval_pred: compute_metrics(eval_pred, tokenizer),
            callbacks=[
                WandbLoggingCallback(fold_num + 1),
                EarlyStoppingCallback(early_stopping_patience=3)
            ],
        )

        # Train the model
        trainer.train()

        # Final evaluation for the fold
        eval_results = trainer.evaluate()
        print(f"Fold {fold_num + 1} results: {eval_results}")

        # Store the final evaluation results for this fold
        fold_metrics.append(eval_results)

        # Predict on the test dataset
        predictions = trainer.predict(test_dataset)

    
        # Decode predictions and save to file
        pred_ids = np.argmax(predictions.predictions[0], axis=-1)
        predicted_texts = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
        true_texts = tokenizer.batch_decode(predictions.label_ids, skip_special_tokens=True)
        input_ids = test_dataset["input_ids"]
        input_texts = tokenizer.batch_decode(input_ids, skip_special_tokens=True)

        # Save predictions to a file

        output_predictions_file = f"{filenamebis}_fold_{fold_num+1}.jsonl"
        with open(output_predictions_file, "w", encoding="utf-8") as writer:
            for input_text, true_text, predicted_text in tqdm(zip(input_texts, true_texts, predicted_texts), total=len(true_texts)):
                output_dict = {
                    "input": input_text,
                    "true": true_text,
                    "predicted": predicted_text
                }
                writer.write(json.dumps(output_dict, ensure_ascii=False) + "\n")
        
        print(f"Predictions for fold {fold_num + 1} saved to {output_predictions_file}")

    # Calculate and log average metrics
    avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics]) for key in fold_metrics[0]}
    wandb.log({f"Averages/{key}": value for key, value in avg_metrics.items()})
    print("Average metrics over all folds:", avg_metrics)

    wandb.finish()

    #merge all to a single file for predcitions

    with open(filename, "w", encoding="utf-8") as merged_file:
        for fold_num in range(len(folds)):
            fold_file = f"{filenamebis}_fold_{fold_num+1}.jsonl"
            if os.path.exists(fold_file):
                with open(fold_file, "r", encoding="utf-8") as f:
                    for line in f:
                        merged_file.write(line)
    print(f"Final predicitons for all folds foudn in {filename}")


In [15]:
cross_val_train(modelname="t5-small", run_name="T5_SMALL_TOP5DOCS_TFIDF",fold_filename="training_top10_qulac_en_tfidf_CLEANED_rep_PREPROCESSED_FOR_MODEL.json",filename="crossval_T5_SMALL_TOP5DOCS_TFIDF.jsonl", 
                tokenizer_global=tokenizer_global, tokenized_dataset_global=tokenized_dataset_global, 
                folds=folds, nb_epochs=30)

Processing Fold 1


Map:   0%|          | 0/599 [00:00<?, ? examples/s]

Recuperation du Fichier FOLD0_training_top5_qulac_en_tfidf_CLEANED_rep_PREPROCESSED_FOR_MODEL.json pour le training du Fold 0


All TF 2.0 model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_8374/4125602158.py:59: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor
1,1.443900,0.798685,0.128292,0.398007,0.179725,0.386804,0.326892
2,0.747400,0.635106,0.148264,0.415373,0.203379,0.406510,0.362502
3,0.612900,0.554118,0.173760,0.464859,0.221757,0.456705,0.409950
4,0.527100,0.532810,0.187899,0.480551,0.235226,0.469740,0.425820
5,0.505400,0.522377,0.191176,0.482766,0.237267,0.474386,0.435280
6,0.476400,0.513959,0.181937,0.478732,0.229788,0.469182,0.430597
7,0.448300,0.511224,0.194721,0.484278,0.243391,0.476282,0.446110
8,0.453400,0.508367,0.191117,0.479950,0.239953,0.470527,0.437823
9,0.453000,0.504915,0.199213,0.488394,0.247031,0.480011,0.446302
10,0.429500,0.505179,0.201045,0.485842,0.248044,0.476194,0.444705


...

Fold 5 results: {'eval_loss': 0.4626505672931671, 'eval_bleu': 0.19004817514984165, 'eval_rouge1': 0.5052960018772827, 'eval_rouge2': 0.22346352779872528, 'eval_rougeL': 0.49198371599245694, 'eval_meteor': 0.44750932540482385, 'eval_runtime': 2.5222, 'eval_samples_per_second': 47.182, 'eval_steps_per_second': 5.947, 'epoch': 20.0}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119/119 [00:00<00:00, 65219.15it/s]

Predictions for fold 5 saved to crossval_T5_SMALL_TOP5DOCS_TFIDF_fold_5.jsonl
Average metrics over all folds: {'eval_loss': 0.47072420716285707, 'eval_bleu': 0.19487567940440836, 'eval_rouge1': 0.4948780064641406, 'eval_rouge2': 0.24259299948389632, 'eval_rougeL': 0.48279901161734956, 'eval_meteor': 0.4506840394118282, 'eval_runtime': 2.2061, 'eval_samples_per_second': 54.5732, 'eval_steps_per_second': 6.8316, 'epoch': 18.0}


Averages/epoch,▁
Averages/eval_bleu,▁
Averages/eval_loss,▁
Averages/eval_meteor,▁
Averages/eval_rouge1,▁
Averages/eval_rouge2,▁
Averages/eval_rougeL,▁
Averages/eval_runtime,▁
Averages/eval_samples_per_second,▁
Averages/eval_steps_per_second,▁
Fold_1/eval/epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇███


Final predicitons for all folds foudn in crossval_T5_SMALL_TOP5DOCS_TFIDF.jsonl


# TEST of top10 TFIDF INSTEAD OF TOP 5

In [18]:
cross_val_train(modelname="t5-small", run_name="T5_SMALL_TOP10DOCS_TFIDF",filename="crossval_T5_SMALL_TOP10DOCS_TFIDF.jsonl", 
                tokenizer_global=tokenizer_global, tokenized_dataset_global=tokenized_dataset_global, 
                folds=folds, nb_epochs=30)

Processing Fold 1


Map:   0%|          | 0/599 [00:00<?, ? examples/s]

Recuperation du Fichier FOLD0_training_top10_qulac_en_tfidf_CLEANED_rep_PREPROCESSED_FOR_MODEL.json pour le training du Fold 0


W0000 00:00:1746914173.151919   18289 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
All TF 2.0 model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_18289/1707016551.py:59: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to th

Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor
1,1.270000,0.777021,0.133553,0.389895,0.188643,0.382010,0.328259
2,0.721600,0.617617,0.151810,0.421219,0.205314,0.410922,0.372651
3,0.602700,0.552534,0.173743,0.458097,0.218951,0.451483,0.406900
4,0.527500,0.529759,0.182090,0.476399,0.233983,0.465897,0.425703
5,0.501300,0.521005,0.185558,0.482259,0.233757,0.471428,0.431041
6,0.471700,0.513119,0.184655,0.482775,0.235356,0.472689,0.437127
7,0.448800,0.511730,0.188841,0.491928,0.240126,0.480332,0.443843
8,0.446300,0.505437,0.196351,0.487957,0.249770,0.477195,0.445810
9,0.449100,0.503571,0.197003,0.490868,0.245304,0.477374,0.443795
10,0.425900,0.501816,0.195673,0.493893,0.248153,0.481236,0.447750


...

Fold 5 results: {'eval_loss': 0.46521592140197754, 'eval_bleu': 0.1981122765840142, 'eval_rouge1': 0.5118333330408786, 'eval_rouge2': 0.23402551947507522, 'eval_rougeL': 0.5000489220240332, 'eval_meteor': 0.4551551575491318, 'eval_runtime': 2.5417, 'eval_samples_per_second': 46.819, 'eval_steps_per_second': 5.902, 'epoch': 20.0}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119/119 [00:00<00:00, 44485.04it/s]

Predictions for fold 5 saved to crossval_T5_SMALL_TOP10DOCS_TFIDF_fold_5.jsonl
Average metrics over all folds: {'eval_loss': 0.4691967606544495, 'eval_bleu': 0.19577301397006885, 'eval_rouge1': 0.499544828190538, 'eval_rouge2': 0.24715247447005922, 'eval_rougeL': 0.48762726879887647, 'eval_meteor': 0.4539207845136238, 'eval_runtime': 2.2192600000000002, 'eval_samples_per_second': 54.2562, 'eval_steps_per_second': 6.792, 'epoch': 19.0}


Averages/epoch,▁
Averages/eval_bleu,▁
Averages/eval_loss,▁
Averages/eval_meteor,▁
Averages/eval_rouge1,▁
Averages/eval_rouge2,▁
Averages/eval_rougeL,▁
Averages/eval_runtime,▁
Averages/eval_samples_per_second,▁
Averages/eval_steps_per_second,▁
Fold_1/eval/epoch,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇███


Final predicitons for all folds foudn in crossval_T5_SMALL_TOP10DOCS_TFIDF.jsonl


# TEST TOP 5 DOC WITH LDA representation(top 3 topics, 6 words per topic)

In [12]:
cross_val_train(modelname="t5-small", run_name="T5_SMALL_TOP5DOCS_LDA",fold_filename="training_top5_qulac_en_LDA_rep_PREPROCESSED_FOR_MODEL.json",filename="crossval_T5_SMALL_TOP5DOCS_LDA.jsonl", 
                tokenizer_global=tokenizer_global, tokenized_dataset_global=tokenized_dataset_global, 
                folds=folds, nb_epochs=30)

Processing Fold 1


Map:   0%|          | 0/599 [00:00<?, ? examples/s]

Recuperation du Fichier FOLD0_training_top5_qulac_en_LDA_rep_PREPROCESSED_FOR_MODEL.json pour le training du Fold 0


W0000 00:00:1746998759.694531    3576 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
All TF 2.0 model weights were used when initializing T5ForConditionalGeneration.

Some weights of T5ForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_3576/3001149367.py:59: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the

Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Rougel,Meteor
1,21.841900,7.619828,0.017142,0.379858,0.174562,0.363956,0.211098
2,11.342300,0.907027,0.116155,0.376647,0.165702,0.361249,0.298095
3,2.456500,0.821207,0.128128,0.397227,0.181379,0.382465,0.320607
4,1.241000,0.786604,0.129720,0.408453,0.184315,0.396296,0.337304
5,1.241000,0.726773,0.137507,0.422170,0.193836,0.409772,0.357320
6,0.868100,0.679802,0.141635,0.425540,0.202337,0.414602,0.358842
7,0.744400,0.613774,0.148365,0.431258,0.202982,0.420614,0.364893
8,0.644300,0.582682,0.168106,0.458259,0.221986,0.447450,0.394347
9,0.644300,0.560229,0.171907,0.471660,0.223225,0.459810,0.412100
10,0.584400,0.551621,0.178832,0.473128,0.225187,0.463300,0.416329


...

Fold 5 results: {'eval_loss': 0.4866338074207306, 'eval_bleu': 0.19189267017712466, 'eval_rouge1': 0.49969230881278925, 'eval_rouge2': 0.2310091500467428, 'eval_rougeL': 0.48913889814836475, 'eval_meteor': 0.448563693788046, 'eval_runtime': 1.3225, 'eval_samples_per_second': 89.983, 'eval_steps_per_second': 3.025, 'epoch': 30.0}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119/119 [00:00<00:00, 83159.31it/s]

Predictions for fold 5 saved to crossval_T5_SMALL_TOP5DOCS_LDA_fold_5.jsonl
Average metrics over all folds: {'eval_loss': 0.491108649969101, 'eval_bleu': 0.1826410387391856, 'eval_rouge1': 0.48551087208932076, 'eval_rouge2': 0.23197845636750708, 'eval_rougeL': 0.47581059241658236, 'eval_meteor': 0.4371465470271202, 'eval_runtime': 1.6092600000000001, 'eval_samples_per_second': 77.73400000000001, 'eval_steps_per_second': 2.5962, 'epoch': 30.0}


Averages/epoch,▁
Averages/eval_bleu,▁
Averages/eval_loss,▁
Averages/eval_meteor,▁
Averages/eval_rouge1,▁
Averages/eval_rouge2,▁
Averages/eval_rougeL,▁
Averages/eval_runtime,▁
Averages/eval_samples_per_second,▁
Averages/eval_steps_per_second,▁
Fold_1/eval/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇████


Final predicitons for all folds foudn in crossval_T5_SMALL_TOP5DOCS_LDA.jsonl
